<a href="https://colab.research.google.com/github/shubhm253/AI/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


The MNIST dataset consists:
mnist.train: 55000 training images
mnist.validation: 5000 validation images
mnist.test: 10000 test images
Each image is 28 pixels (rows) by 28 pixels (cols). We treat each image as a sequence of data, that is, the first row is the first step; second row is the second step and so on. Therefore, n_steps = number of rows and n_inputs = number of columns.

In [0]:
import tensorflow as tf

The mnist dataset from TensorFlow assumes that you are using one-hot encoding, however, we are not going to do that. Therefore, we need to reshape the dataset from [num_data, 28*28] to [num_data, n_steps, n_inputs]. Since there are many outputs from the RNN, we only care about the last one. As a result, “state” in the code is considered as our output.

In [5]:
# hyperparameters
n_neurons = 128
learning_rate = 0.001
batch_size = 128
n_epochs = 10
# parameters
n_steps = 28 # 28 rows
n_inputs = 28 # 28 cols
n_outputs = 10 # 10 classes
# build a rnn model
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
logits = tf.layers.dense(state, n_outputs)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure to update network weights iterative based in training data.

Reshape the test dataset to [num_test, n_steps, n_inputs]

In [0]:
# input data
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets(“MNIST_data/”)
X_test = mnist.test.images # X_test shape: [num_test, 28*28]
X_test = X_test.reshape([-1, n_steps, n_inputs])
y_test = mnist.test.labels

In [8]:
# initialize the variables
init = tf.global_variables_initializer()
# train the model
with tf.Session() as sess:
    sess.run(init)
    n_batches = mnist.train.num_examples // batch_size
    for epoch in range(n_epochs):
        for batch in range(n_batches):
            X_train, y_train = mnist.train.next_batch(batch_size)
            X_train = X_train.reshape([-1, n_steps, n_inputs])
            sess.run(optimizer, feed_dict={X: X_train, y: y_train})
        loss_train, acc_train = sess.run(
            [loss, accuracy], feed_dict={X: X_train, y: y_train})
        print('Epoch: {}, Train Loss: {:.3f}, Train Acc: {:.3f}'.format(
            epoch + 1, loss_train, acc_train))
    loss_test, acc_test = sess.run(
        [loss, accuracy], feed_dict={X: X_test, y: y_test})
    print('Test Loss: {:.3f}, Test Acc: {:.3f}'.format(loss_test, acc_test))

Epoch: 1, Train Loss: 0.264, Train Acc: 0.898
Epoch: 2, Train Loss: 0.239, Train Acc: 0.930
Epoch: 3, Train Loss: 0.182, Train Acc: 0.938
Epoch: 4, Train Loss: 0.062, Train Acc: 0.977
Epoch: 5, Train Loss: 0.061, Train Acc: 0.992
Epoch: 6, Train Loss: 0.089, Train Acc: 0.969
Epoch: 7, Train Loss: 0.070, Train Acc: 0.984
Epoch: 8, Train Loss: 0.074, Train Acc: 0.969
Epoch: 9, Train Loss: 0.033, Train Acc: 0.992
Epoch: 10, Train Loss: 0.043, Train Acc: 0.992
Test Loss: 0.098, Test Acc: 0.970
